# Querying

This notebook demonstrates how to retrieve, query and search data using the Forge.

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

## Retrieval

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
resource = forge.retrieve(jane.id)

In [ ]:
resource == jane

### specific version

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
forge.tag(jane, "v1")

In [ ]:
jane.email = "jane.doe@epfl.ch"

In [ ]:
forge.update(jane)

In [ ]:
try:
    # DemoStore
    print(jane._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane._store_metadata._rev)

In [ ]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [ ]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [ ]:
jane_v1 == jane_v1_tag

### crossbucket retrieval
It is possible to retrieve resources stored in buckets different then the configured one. The configured store should of course support it.

In [ ]:
resource = forge.retrieve(jane.id, cross_bucket=True) # cross_bucket defaults to False

### error handling

In [ ]:
resource = forge.retrieve("123")

In [ ]:
resource is None

## Searching

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
john = Resource(type="Person", name="John Smith")

In [ ]:
# association_jane = Resource(type="Association", agent=jane)
association_jane = Resource(type="Dataset", contribution=jane)

In [ ]:
# association_john = Resource(type="Association", agent=john)
association_john = Resource(type="Dataset", contribution=john)

In [ ]:
associations = [association_jane, association_john]

In [ ]:
forge.register(associations)

The `paths` method load the data structure for the given type.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about modeling and types.

In [ ]:
# p = forge.paths("Association")
p = forge.paths("Dataset")

You have autocompletion on `p` and this can be used to build a search.

Note: There is a known issue for RdfModel which requires using `p.type.id` instead of `p.type`.

In [ ]:
# resources = forge.search(p.type == "Association")

resources = forge.search(p.type.id=="Dataset", limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

In [ ]:
forge.as_dataframe(resources, store_metadata=True)

### crossbucket search
It is possible to search for resources stored in buckets different then the configured one. The configured store should of course support it.

In [ ]:
resources = forge.search(p.type.id == "Dataset", limit=10, cross_bucket=True)  # cross_bucket defaults to False

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

In [ ]:
#Furthermore it is possible to filter by bucket when cross_bucket is set to True. Setting a bucket value when cross_bucket is False will trigger a not_supported exception.
resources = forge.search(p.type.id == "Dataset", limit=3, cross_bucket=True, bucket=<str>)  # add a bucket

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

### nested field querying

You have autocompletion on `p` but also on nested properties like `p.agent`.

Note: There is a known issue for RdfModel which prevents from searching on the name.

In [ ]:
# resources = forge.search(p.type == "Association", p.agent.name == "John Smith")
resources = forge.search(p.type.id == "Dataset", p.contribution.type == "Person", limit=3)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

## Graph traversing

SPARQL is used as a query language.

A SPARQL query rewriting strategy lets users write simplified queries without using prefix declarations, prefix names or long IRIs. With this strategy, the user could only provides type and property names. For a given entity, these names could be seen in its template.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates.

Note: DemoStore doesn't implement SPARQL operations yet. Please use another store for this section.

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
john = Resource(type="Person", name="John Smith")

In [ ]:
# association = Resource(type="Association", agent=[jane, john])
association = Resource(type="Dataset", contribution=[jane, john])

In [ ]:
forge.register(association)

In [ ]:
# forge.template("Association")
forge.template("Dataset")

In [ ]:
# query = """
#     SELECT ?id ?name
#     WHERE {
#         ?id a Association ;
#             agent ?agent .
#         ?agent name ?name .
#     }
# """
query = """
    SELECT ?id ?p ?o
    WHERE {
        ?id a Dataset ;
        contribution ?contributor.
        ?contributor name ?name.
    }
"""

In [ ]:
resources = forge.sparql(query, limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
type(resources[0])

In [ ]:
forge.as_dataframe(resources)

### rewritten query display

In [ ]:
resources = forge.sparql(query, limit=3, debug=True)

## Downloading

Note: DemoStore doesn't implement file operations yet. Please use another store for this section.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
! ls -p ../../data | egrep -v /$

In [ ]:
distribution = forge.attach("../../data")

In [ ]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [ ]:
forge.register(association)

In [ ]:
forge.download(association, "distribution.contentUrl", "./downloaded/")

In [ ]:
! ls -l ./downloaded/

In [ ]:
# ! rm -R ./downloaded/